In [1]:
!pip install pytorch_lightning
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.0/719.0 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.3/269.3 kB 27.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 39.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.8/198.8 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
import torch.utils.data as data
import zipfile

# Mount Google Drive to access the dataset
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Set the path to your zipped dataset
zip_file_path = '/content/drive/MyDrive/M.Tech IITM/Deep Learning /aksharantar_sampled.zip'

# Extract the dataset to a folder in Google Drive
zip_ref = zipfile.ZipFile(zip_file_path, 'r')
zip_ref.extractall('aksharantar_sampled_extracted')
zip_ref.close()

In [4]:
import torch
import torch.nn as nn
from google.colab import drive
import csv


# Path to your CSV file on Google Drive (Extracted file)
csv_file_path = '/content/aksharantar_sampled_extracted/aksharantar_sampled/hin/hin_train.csv'

# Read the CSV file and extract the sequence of characters
with open(csv_file_path, 'r') as f:
    reader = csv.reader(f)
    chars = []
    for row in reader:
        chars.extend(row[0])  # assuming the first column of the CSV file contains the text data

charS=set(chars)
charS.add('|')
char_set = list(charS)


# Define the mapping between characters and integer indices
char_to_idx_latin= {char: i+1 for i, char in enumerate(char_set)}


In [5]:
max_length_devanagari=0
with open(csv_file_path, 'r') as f:
    reader = csv.reader(f)
    chars = []

    for row in reader:
        chars.extend(row[1])  # assuming the first column of the CSV file contains the text data

# Define the character set
charS=set(chars)
charS.add('|')
char_set = list(charS)

# Define the mapping between characters and integer indices
char_to_idx_lang ={char: i+1 for i, char in enumerate(char_set)}

In [6]:
max_length_latin = 0

# Read the CSV file and extract the sequence of characters
with open(csv_file_path, 'r') as f:
    reader = csv.reader(f)
    chars = []

    for row in reader:
        length = len(row[0])  # assuming the column you want to check is the first one
        if length > max_length_latin:
            max_length_latin = length

In [7]:
# max_length_latin

In [8]:
max_length_devanagari = 0
# Read the CSV file and extract the sequence of characters
with open(csv_file_path, 'r') as f:
    reader = csv.reader(f)
    chars = []

    for row in reader:
        length = len(row[1])  # assuming the column you want to check is the first one
        if length > max_length_devanagari:
            max_length_devanagari = length

In [9]:
# char_to_idx_lang

In [10]:
# char_set

In [11]:
# char_to_idx_latin

In [12]:
# max_length_devanagari

In [13]:
def word_to_indices(word, max_length,dict):
    # Convert the characters to integer indices using the char_to_idx mapping
    indices = [dict.get(c, -1) for c in word]
    # Filter out characters not in the dictionary
    indices = [idx for idx in indices if idx >= 0]
    # Add padding if necessary to make the sequence length equal to max_length
    if len(indices) < max_length:
        indices += [0] * (max_length - len(indices))
    # Truncate the sequence if necessary to make the sequence length equal to max_length
    elif len(indices) > max_length:
        indices = indices[:max_length]
    # Add the start and end tokens to the sequence
    indices = [dict['|']] + indices + [dict['|']]
    # Convert the list of indices to a tensor
    tensor = torch.tensor(indices)
    return tensor
pairs=[]

# Read the CSV file containing Latin-Devanagari word pairs
with open('/content/aksharantar_sampled_extracted/aksharantar_sampled/hin/hin_train.csv', 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        latin_word = row[0]
        devanagari_word = row[1]
        # Convert the Latin word to a tensor of integer indices
        latin_indices = word_to_indices(latin_word, max_length_latin,char_to_idx_latin)
        devanagari_indices= word_to_indices(devanagari_word,max_length_devanagari ,char_to_idx_lang)
        pairs.append([latin_indices,devanagari_indices])
        # print(devanagari_word, devanagari_indices)
        # print(latin_word,latin_indices)
        # print('**************************************************')


In [14]:
# # len(train_dataloader[0])
# for x,y in train_dataloader:
#   print(x.shape,y.shape)
#   break

In [15]:
train_dataloader = torch.utils.data.DataLoader(pairs, batch_size=64, shuffle=True)

In [31]:
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = nn.Embedding(input_dim, hidden_dim)
        self.rnn = nn.GRU(hidden_dim, hidden_dim)

    def forward(self, x):
        embedded = self.embedding(x)
        output, hidden = self.rnn(embedded)
        return hidden

class Decoder(nn.Module):
    def __init__(self, output_dim, hidden_dim):
        super().__init__()
        self.hidden_dim = hidden_dim
        # print("Out embed",hidden_dim)
        self.embedding = nn.Embedding(hidden_dim, hidden_dim)
        self.rnn = nn.GRU(hidden_dim, hidden_dim)
        # print("Out Dime",output_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x, hidden):
        # print("hidden",hidden.shape)
        # print("Bef x",x.shape)
        x = x.unsqueeze(0)
        # print("Aft x",x.shape)
        embedded = self.embedding(x)
        # print("embedded",embedded.shape)
        # print("hidden",hidden.shape)
        output, hidden = self.rnn(embedded, hidden)
        prediction = self.fc(output.squeeze(0))
        return prediction, hidden

class Seq2Seq(pl.LightningModule):
    def __init__(self, input_dim, output_dim, hidden_dim):
        super().__init__()
        self.encoder = Encoder(input_dim, hidden_dim)
        self.decoder = Decoder(output_dim, hidden_dim)
        self.teacher_forcing_ratio = 0.5

    def forward(self, src, trg):############################################Where r we calling this forward from
        # print("Forward")
        # print("src",src.shape,"trg",trg.shape)
        batch_size = trg.shape[0]###################################################################
        # print("src",src.shape,"trg",trg.shape)
        max_len = trg.shape[1]
        # print("src",src.shape,"trg",trg.shape)
        trg_vocab_size = self.decoder.fc.out_features
        # print("src",src.shape,"trg",trg.shape)
        src = src.transpose(0,1)
        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)
        hidden = self.encoder(src)
        # print("trg",trg.shape)
        # input = trg[0,:]
        input = trg[:,0]
        # print("INP,Dec",input.shape)
        for t in range(1, max_len):
            # print("t",t,max_len,input.shape)
            output, hidden = self.decoder(input, hidden)
            # print("Out",outputs.shape,output.shape)
            outputs[t] = output
            top1 = output.argmax(1)
            input = top1 if self.teacher_forcing_ratio > torch.rand(1).item() else trg[:,t]
        return outputs

    def training_step(self, batch, batch_idx):
        src, trg = batch
        # print("Before forward trg",trg.shape)
        output = self(src, trg)
        # print("After train output",output.shape,"trg",trg.shape)
        # After train output torch.Size([22, 64, 67]) trg torch.Size([64, 22])
        # torch.Size([26, 2, 71]) torch.Size([2, 26])
        
#Ashok 
        # rows = np.arange(67)
        # expected = torch.zeros(size=(output.shape))
        # rows = np.arange(len(output))
        # for f in range(len(trg)):
        #     col = trg[f]
        #     expected[rows,f,np.array(col.cpu())] = 1
#
 
#         
        cols = torch.arange(output.shape[1]).unsqueeze(1)
        rows = torch.arange(output.shape[0])
        expected = torch.zeros(size=output.shape)
        expected[rows, cols, trg.cpu()] = 1


        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        expected = expected[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        loss = self.loss_fn(output, expected)
        self.log('train_loss', loss)
        return {'loss': loss}

    def validation_step(self, batch, batch_idx):
        # print("Validation")
        src, trg = batch
        output = self(src, trg)
        # print("Validation",output.trg.shape)
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        loss = self.loss_fn(output, trg)
        self.log('val_loss', loss)

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters())
        return optimizer

    def loss_fn(self, output, trg):
        # mask = (trg != self.pad_idx).float()
        criterion = nn.CrossEntropyLoss()
        # print("Loss cal",output.shape,trg.shape)
        loss = criterion(output, trg)
        # loss = criterion(output, trg) * mask
        return loss.mean()


In [32]:
model = Seq2Seq(len(char_to_idx_latin)+2, len(char_to_idx_lang)+2, 256)

In [33]:
trainer = pl.Trainer(max_epochs=1)
# for x,y in train_dataloader:
  # print(x.shape,y.shape)
  # break
trainer.fit(model, train_dataloader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name    | Type    | Params
------------------------------------
0 | encoder | Encoder | 402 K 
1 | decoder | Decoder | 477 K 
------------------------------------
879 K     Trainable params
0         Non-trainable params
879 K     Total params
3.519     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.


In [ ]:
  # max_length_latin

In [ ]:
# for x,y in train_dataloader:
#   print(y)
#   break